In [ ]:
import numpy as np
import matplotlib.image as Image
import matplotlib.pyplot as plt
np.random.seed(42)

In [ ]:
class Creature:
    def __init__(self, canvas, picture, x_pos = None, y_pos = None, px_radius = None, r = None, g = None, b = None, a = None):
        #Fix Parameters
        self.picture = picture
        self.canvas = canvas
        self.new_canvas = canvas.copy()
        #self.new_canvas.setflag(write=1)
        self.h, self.w, _ = canvas.shape
        self.x_pos = x_pos or np.random.randint(0, self.w)
        self.y_pos = y_pos or np.random.randint(0, self.w)
        self.px_radius = px_radius or np.random.randint(1,min(self.h, self.w))
        self.r = r or np.random.randint(0,255, dtype=np.uint8)
        self.g = g or np.random.randint(0,255, dtype=np.uint8)
        self.b = b or np.random.randint(0,255, dtype=np.uint8)
        self.a = a or np.random.randint(0,255, dtype=np.uint8)

        #Type-Casting
        self.r = np.uint8(self.r)
        self.g = np.uint8(self.g)
        self.b = np.uint8(self.b)
        self.a = np.uint8(self.a)
        
        #Calculate Mask
        Y, X = np.ogrid[:self.h, :self.w]
        dist_from_center = np.sqrt((X - self.x_pos)**2 + (Y - self.y_pos)**2)
        self.mask = dist_from_center <= self.px_radius
        
        #Calculate new Canvas
        orig = self.new_canvas[self.mask]
        new = np.ones_like(orig)*(self.r,self.g,self.b)
        self.new_canvas[self.mask] = np.array(orig*(255-self.a)+new*self.a, dtype=np.uint8)
        
        #Evaluate fitness
        mse = np.sum(np.square(self.picture - self.new_canvas)) / (self.h * self.w)
        self.fitness = 100/(1+mse)
        
    def getNewCanvas(self):
        return self.new_canvas
    
    def getFitness(self):
        return self.fitness
        
    def printCreature(self):
        plt.imshow(self.new_canvas)
        plt.title(f'Fitness {self.fitness}')
        plt.show()
        
    def getGenes(self):
        return [self.x_pos, self.y_pos, self.px_radius, self.r, self.g, self.b, self.a]

In [ ]:
def softmax(x):
    x = np.array(x)
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
path = './pictures/cloud2.jpg'

In [ ]:
img = Image.imread(path)

In [ ]:
canvas = np.ones_like(img)*255

In [ ]:
max_population = 100
num_epochs = 50
num_iterations = 500
mutation_rate = 0.02

In [ ]:
for e in range(num_epochs):
    population = [Creature(canvas, img) for _ in range(max_population)]
    
    for i in range(num_iterations):
        fitness = [c.getFitness() for c in population]
        best_fitness = list(softmax(fitness))
        father = np.random.choice(population,p=best_fitness)
        mother = np.random.choice(population,p=best_fitness)
        replace_index = np.argmin(best_fitness)
        father_genes = father.getGenes()
        mother_genes = mother.getGenes()
        intersect = np.random.randint(1,len(father_genes)-1)
        child_genes = father_genes[:intersect] + mother_genes [intersect:]
        for idx, g in enumerate(child_genes):
            if np.random.rand() <= mutation_rate:
                child_genes[idx] = np.random.randint(0,max(1,2*g))

        child = Creature(father.canvas, img, *child_genes)
        population[replace_index] = child
        
        #best_fit = 0
        #for p in population:
        #    f = p.getFitness()
        #    if f > best_fit:
        #        best_fit = f
        #        best = p
        #print(f'Best Creature in iter {i} / epo {e}:')
        #p.printCreature()
    
    #Find best Creature
    best_fit = 0
    for p in population:
        f = p.getFitness()
        if f > best_fit:
            best_fit = f
            best = p
    
    #set canvas
    canvas = best.getNewCanvas()
    
    print(f'Epoch {e}')
    best.printCreature()